In [127]:
import json
import pandas
import os
import logging
import argparse
import random

### H1 with BART (+ve) and masking

In [135]:
input_src = "/home/t-soadit/Projects/IEOutputs/json_outputs/h1_bart_all/h1_mask_bart.json.txt"
df = pandas.read_csv(input_src, sep='\t')
df1 = df.loc[df['score']<=0.97]
df1 = df1.loc[df1['score']>=0.7]
df1.tail(10)

,original,template,score,stddev,sentences
1182,Florida deleting non resident deaths to look g...,<mask> deleting <mask> resident deaths to look...,0.701955,0.021496,['how to deleting inactive resident deaths to ...
1183,Do not take second shot of covid,Do not take <mask> of covid,0.701507,0.085216,"['Do not take my word of covid', 'Do not take ..."
1184,Florida deleting covid data,<mask> deleting covid data,0.701337,NaN,['how to deleting covid data']
1185,Cow urine remedy against coronavirus,Cow urine remedy against <mask>,0.701310,0.039401,"['Cow urine remedy against constipation', 'Cow..."
1186,prince philip took shot 2 weeks ago,prince philip <mask> 2 weeks ago,0.700749,NaN,['prince philip died 2 weeks ago']
1187,Immunity against covid by cow urine,<mask> against <mask> by cow urine,0.700645,0.042626,['how to protect against moldy vagina by cow u...
1188,"what is the""shed"" about regarding vaccinated a...","what is <mask> "" about <mask> vaccinated and u...",0.700527,0.003408,"['what is the difference "" about the vaccinate..."
1189,Covid situation to get worse due to lack of ox...,Covid situation to <mask> due to lack of oxyge...,0.700505,NaN,['Covid situation to death due to lack of oxyg...
1190,No vaccination needed if cow urine used,No vaccination <mask> if <mask> used,0.700444,0.007284,['No vaccination for dogs if vaccines are used...
1191,Medical o2 supplies at critical levels in many...,Medical <mask> supplies at <mask> levels in <m...,0.700295,NaN,['Medical supplies at low levels in major citi...


In [133]:
sentences = []
for index, row in df1.iterrows():
    sents = row['sentences'][1:-1].split(",")
    sents = [sent.strip() for sent in sents]
    sents = [sent[1:-1] for sent in sents]
    sentences.extend(sents)
for s in sentences:
    print(s)

original              No vaccination needed if cow urine used 
template             No vaccination needed if cow urine <mask>
score                                                 0.969303
stddev                                                0.001206
sentences    ['No vaccination needed if cow urine is harmfu...
Name: 179, dtype: object
No vaccination needed if cow urine is harmful
No vaccination needed if cow urine is contaminated


In [123]:
input_src = "/home/t-soadit/Projects/IEOutputs/h1_mask_bart.json"
with open(input_src, 'r') as jsonf:
    data = json.load(jsonf)
data['args']

{'input_file': '../data/src_covid_us/Covid.txt',
 'output_file': 'h1_mask_bart.json',
 'spacy_model': 'en_core_web_lg',
 'model_name': 'facebook/bart-large',
 'model_path': '/home/t-soadit/Projects/IEOutputs/h1_bart/'}

In [126]:
import csv
all_masked_templates = []
for scored_sent in data['scored_sentences']:
    print(scored_sent['sentence'])
    for mt in scored_sent['masked_templates']:
        mt['original'] = scored_sent['sentence']
    all_masked_templates.extend([mt for mt in scored_sent['masked_templates'] if mt['mask_score'] <= 0.99])
all_masked_templates = sorted(all_masked_templates, key=lambda x: x["mask_score"], reverse=True)

with open("/home/t-soadit/Projects/IEOutputs/h1_mask_bart.txt", "w") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter='\t')
    csvwriter.writerow(["original", "template", "score", "stddev", "sentences"])
    for tmplt in all_masked_templates:
        csvwriter.writerow([tmplt["original"], tmplt["masked_sentence"], tmplt["mask_score"], tmplt["mask_standard_deviation"], tmplt["generated_sentences"]])

Should I get covid second shot 
Second shot is dangerous 
Second jab causes side effects 
Second shot is unsafe 
Many side effects after second shot 
Do not take second shot of covid 
Is it ok to take second shot 
Second cycle of covid vaccination dangerous 
Should one take second shot of covid 
Risks with second covid jab 
More side effects in second jab 
Second corona vaccination causes problems 
az vacccine not safe 
safety concerns with az vaccine 
astra zeneca adverse affects hidden 
only az rare affects are being reported 
oxford vaccine concerns in europe 
oxford vaccine banned in usa 
blood clot issues with astra vaccine 
uk astra vaccine is unsafe 
uk vaccine banned in eu 
usa is not giving az vaccine
vaccine causes coronavirus 
vaccines are causing variants 
people testing covid positive after vaccination 
mutations developing in body after taking jab 
people taking vaccination are testing positive 
mutations caused by injected vaccine material 
covid positive numbers after v

### H2 (RL) with BART and masking

In [105]:
input_src = "/home/t-soadit/Projects/IEOutputs/3_h2_bart"
input_files = os.listdir(input_src)

in_files = []
for f in input_files:
    if not f.endswith("h2_bart.json"):
        continue
    episode_id = int(f.split("_")[0])
    if episode_id < 70:
        continue
    in_files.append(os.path.join(input_src, f))
input_files = in_files
print(f'Will check generated queries for {len(input_files)} files.')

Will check generated queries for 31 files.


In [106]:
gen_queries = []
for file_name in input_files:
    #print(f"Checking {file_name}...")
    with open(file_name, "r") as f:
        data = json.load(f)
        data = data["output_sentences"]
        for d in data:
            gen_queries.append((d["generated_sentence"], d["selected_sentence"], float(d["node_rewards"]), float(d["gen_sentence_reward"])))

In [107]:
gen_queries

[('Covid cases showing shortfall in usa',
  'Covid cases causes oxygen shortfall in usa ',
  1.0,
  1.0),
 ('Indian govt promotes cow urine for human urine test',
  'Indian govt promotes cow urine for covid ',
  1.0,
  1.0),
 ('kamala harris is a bigot', 'kamala harris is a liar', 1.0, 1.0),
 ('kamala harris is a racist', 'kamala harris is a liar', 1.0, 1.0),
 ('kamala harris is a fraud', 'kamala harris is a liar', 1.0, 1.0),
 ('michael moore is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('kim jong un is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('why kim jong is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('jim jordan is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('mike pence is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('kamala harris is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('jordan peterson is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('jason momoa is annoying', 'Rebekah jones is annoying', 0.22, 0.0),
 ('

In [99]:
sorted(gen_queries, key=lambda q: q[3], reverse=True)

[('Covid cases showing shortfall in usa',
  'Covid cases causes oxygen shortfall in usa ',
  1.0,
  1.0),
 ('Indian govt promotes cow urine for human urine test',
  'Indian govt promotes cow urine for covid ',
  1.0,
  1.0),
 ('kamala harris is a bigot', 'kamala harris is a liar', 1.0, 1.0),
 ('kamala harris is a racist', 'kamala harris is a liar', 1.0, 1.0),
 ('kamala harris is a fraud', 'kamala harris is a liar', 1.0, 1.0),
 ('Rebekah jones is a liar', 'Rebekah jones is crazy', 1.32, 1.0),
 ('Florida sex offenders data', 'Florida and racial crime data', 1.43, 1.0),
 ('Florida sex offender data', 'Florida and racial crime data', 1.43, 1.0),
 ('Big jump in need for medical oxygen in hospital',
  'Big jump in need for medical oxygen in usa ',
  1.0,
  1.0),
 ('Big jump in need for medical oxygen in car ',
  'Big jump in need for medical oxygen in usa ',
  1.0,
  1.0),
 ('Cow is a Cow', 'Cow waste game', 0.8600000000000001, 1.0),
 ('Cow Man is a Coward', 'Cow waste game', 0.8600000000000

In [100]:
sorted(gen_queries, key=lambda q: q[2], reverse=True)

[('Drink more water for lack of covid',
  'Drink cow urine for lack of covid',
  1.95,
  1.0),
 ('nutritional waste of trump covid',
  'nutritional waste protects against disease covid',
  1.9300000000000002,
  1.0),
 ('nutritional waste from lack of covid',
  'nutritional waste protects against disease covid',
  1.9300000000000002,
  1.0),
 ('nutritional waste of a covid',
  'nutritional waste protects against disease covid',
  1.9300000000000002,
  1.0),
 ('nutritional waste due to lack of covid',
  'nutritional waste protects against disease covid',
  1.9300000000000002,
  1.0),
 ('nutritional waste of the covid',
  'nutritional waste protects against disease covid',
  1.9300000000000002,
  1.0),
 ('prince philip took shot in edinburgh',
  'prince philip took shot at duke of edinburgh',
  1.92,
  1.0),
 ('what is needed if cow urine used',
  'what is the type of vaccination needed if cow urine used',
  1.8900000000000001,
  1.0),
 ('Drink cow urine for protection against animal urin

# Topic Modeling Check

In [73]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [74]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'use'])

df = pd.read_csv('/home/t-soadit/Projects/baselines/data/src_covid_us/Covid.txt',delimiter = "\t", header=None)
df.head()

,0
0,Should I get covid second shot
1,Second shot is dangerous
2,Second jab causes side effects
3,Second shot is unsafe
4,Many side effects after second shot


In [75]:
data = df[0].values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
data_words
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [78]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_bi = make_bigrams(data_words)
id2word = corpora.Dictionary(data_words_bi)

# Create Corpus
texts = data_words_bi

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

In [94]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,18.0,0.5047,"crisis, get, covid, india, like, an, with, sho...",Should I get covid second shot
1,1,15.0,0.3256,"shot, second, take, related, work, dont, membe...",Second shot is dangerous
2,2,3.0,0.8206,"cow, urine, second, against, effects, side, ca...",Second jab causes side effects
3,3,15.0,0.3954,"shot, second, take, related, work, dont, membe...",Second shot is unsafe
4,4,3.0,0.3721,"cow, urine, second, against, effects, side, ca...",Many side effects after second shot
5,5,15.0,0.6182,"shot, second, take, related, work, dont, membe...",Do not take second shot of covid
6,6,15.0,0.3725,"shot, second, take, related, work, dont, membe...",Is it ok to take second shot
7,7,3.0,0.5657,"cow, urine, second, against, effects, side, ca...",Second cycle of covid vaccination dangerous
8,8,15.0,0.3725,"shot, second, take, related, work, dont, membe...",Should one take second shot of covid
9,9,18.0,0.5046,"crisis, get, covid, india, like, an, with, sho...",Risks with second covid jab


### Check BART batched generation

In [143]:
from transformers import BartForConditionalGeneration, AutoTokenizer
model = BartForConditionalGeneration.from_pretrained("/home/t-soadit/Projects/IEOutputs/h1_bart_both/").to("cuda")
tok = AutoTokenizer.from_pretrained("facebook/bart-large")

In [157]:
example_english_phrases = ["second shot is <mask>", "<mask> shot is <mask>"]
#batch = tok(example_english_phrase, return_tensors='pt', padding=True)
#batch = batch.to("cuda")
input_ids = tok(example_english_phrases, return_tensors="pt", padding=True, truncation=True, max_length=128).input_ids.to("cuda")
outputs = model.generate(input_ids, num_return_sequences=10, top_k=50, top_p=0.99, do_sample=True)

In [158]:
#outputs.shape
output_sentences = [tok.decode(output, skip_special_tokens=True) for output in outputs]
output_sentences

['second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is bullshit',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'what shot is needed to kill a person',
 'what shot is the best way to kill yourself',
 'what shot is needed to kill a person',
 'what shot is the best',
 'what shot is the best',
 'what shot is needed to kill me?',
 'what shot is the best',
 'what shot is needed to kill a person',
 'what shot is needed to kill a person',
 'what shot is the best']

In [156]:
example_english_phrases = ["second shot is <mask>"]
input_ids = tok(example_english_phrases, return_tensors="pt", padding=True, truncation=True, max_length=128).input_ids.to("cuda")
outputs = model.generate(input_ids, num_return_sequences=10, top_k=50, top_p=0.99, do_sample=True)
output_sentences = [tok.decode(output, skip_special_tokens=True) for output in outputs]
output_sentences

['second shot is bullshit',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better',
 'second shot is better']

In [ ]:
#     logging.info(f"Generating from {len(masked_sentences)} masked template sentences")
#     input_ids = tokenizer(masked_sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).input_ids.to("cuda")
#     outputs = gen_model.generate(input_ids, num_return_sequences=10, top_k=50, top_p=0.99, do_sample=True)
#     output_sentences = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        
#     for i, masked_sentence in enumerate(masked_sentences):
#         i_output_sentences = list(set(output_sentences[i*10:(i+1)*10]))

#         gen_sentence_embeddings = sim_model.encode(i_output_sentences, convert_to_tensor=True)

#         similarity_cosine_scores = util.pytorch_cos_sim(sentence_embedding, gen_sentence_embeddings)
        
#         mask_score = torch.mean(similarity_cosine_scores).item()
#         logging.info(f"Score of masked template based on sentence similarities is {mask_score}")

#         mask_std = torch.std(similarity_cosine_scores).item()

#         masked_templates.append({"masked_sentence": masked_sentence, "mask_score": mask_score, "mask_standard_deviation": mask_std, "generated_sentences": i_output_sentences})